<a href="https://colab.research.google.com/github/bjin2364/mit-deep-learning/blob/main/Building_an_MNIST_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NOTE: Click the top-left "Open In Playground" button! 

In [1]:
# install basical image libs
!pip install Pillow>=5.0.0
!pip install -U image

# install torch and torchvision (a utility library for computer vision that provides many public datasets and pre-trained models)
from os.path import exists
#from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
#platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
#!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.1.0-{platform}-linux_x86_64.whl torchvision

     |████████████████████████████████| 7.9 MB 30.1 MB/s 
  Created wheel for image: filename=image-1.5.33-py2.py3-none-any.whl size=19495 sha256=cc39827bee1f861dc16fa01c01385084f5559e53f94a2818fdcb83af206d4008
  Stored in directory: /root/.cache/pip/wheels/56/88/e6/897194cfe8c08a8b9afd881d3bf53d102e13fa39607d721383
Successfully built image


## Building an MNIST Classifier


Using what we have learned, let's build a simple MNIST digits classifier.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


torch.manual_seed(1234)  # for reproducibility

cuda0 = torch.device('cuda:0')

### Loading Data

The [`torchvision`](https://pytorch.org/docs/stable/torchvision/index.html) library provides a wide range of standard vision datasets and networks with pretrained weights. We will use [the `torchvision.datasets.MNIST` class](https://pytorch.org/docs/stable/torchvision/datasets.html#mnist) to easily access the MNIST dataset.

In [3]:
import torchvision

mnist_train = torchvision.datasets.MNIST(root='./data', download=True, train=True)


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Each element of the dataset is a 2-tuple, consisting of the image of the digit, and its label. E.g.,

In [4]:
image, label = mnist_train[13]
print('This is a digit {}:'.format(label))
image

This is a digit 6:


In [5]:
type(image)

PIL.Image.Image

The image is given as a `PIL.Image`. To automatically obtain a `torch.Tensor`, we can add a [`torchvision.transforms.ToTensor` ](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.ToTensor) transform when constructing the dataset:

In [6]:
mnist_train = torchvision.datasets.MNIST(root='./data', download=True, train=True,
                                         transform=torchvision.transforms.ToTensor())  # the ToTensor transform converts PIL.Image to torch.Tensor

In [7]:
image, label = mnist_train[13]
print(type(image))
print(image.shape)  # MNIST images are 28x28, and have a single channel representing brightness.

<class 'torch.Tensor'>
torch.Size([1, 28, 28])


In [8]:
print('This image has max={}'.format(image.max()), 'and min={}'.format(image.min()))

This image has max=1.0 and min=0.0


In deep learning, it is often a good idea to normalize network inputs to be centered around zero. We use the [`torchvision.transforms.Normalize`](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.Normalize) tranform to achieve this. Adding this transform, we construct the dataset as:

In [9]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=(0.5,), std=(0.5,)),  # [0, 1] range => [-1, 1] range
])

mnist_train = torchvision.datasets.MNIST(root='./data', download=True, train=True,
                                         transform=transform)
mnist_val = torchvision.datasets.MNIST(root='./data', download=True, train=False,
                                         transform=transform)

In [10]:
print('training set size:\t{}'.format(len(mnist_train)))
print('validation set size:\t{}'.format(len(mnist_val)))

training set size:	60000
validation set size:	10000


We use the PyTorch `torch.utils.data.DataLoader` to automatically load batched data. 

In [11]:
batch_size = 512

train_loader = torch.utils.data.DataLoader(mnist_train, 
                                           batch_size=batch_size,
                                           shuffle=True,                   # shuffle training set
                                           num_workers=4,                  # turns on multi-processing loading so training is not blocked by data loading
                                           pin_memory=True)                # pin_memory allows faster transfer from CPU to GPU
val_loader = torch.utils.data.DataLoader(mnist_val, 
                                         batch_size=batch_size, 
                                         num_workers=4, 
                                         pin_memory=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [12]:
# Each element yielded by `train_loader` (a Python iterable) is still a 2-tuple, 
# but now consisting of a batched image tensor, and a batched label tensor.

images, labels = next(iter(train_loader))
print('batched image tensor shape: {}'.format(images.shape))
print('batched label tensor shape: {}'.format(labels.shape))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


batched image tensor shape: torch.Size([512, 1, 28, 28])
batched label tensor shape: torch.Size([512])


### Building the Network

We will use a convolutional network for classification. The following architecture is adapted from the famous [LeNet-5](https://ieeexplore.ieee.org/document/726791) [1].




[1] LeCun, Yann, et al. "Gradient-based learning applied to document recognition." Proceedings of the IEEE 86.11 (1998): 2278-2324.

In [13]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(in_features=16 * 5 * 5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.reshape(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [14]:
net = MyNet().to(cuda0)

In [15]:
# This network output a size 10 vector for each input image, as verified below 
# using a random input tensor.

random_input = torch.randn(123, 1, 28, 28, device=cuda0)
output = net(random_input)
print(output.shape)

torch.Size([123, 10])


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


### Training Loop


For classification, we will use the cross-entropy loss [`F.cross_entropy`](https://pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html?highlight=cross_entropy#torch.nn.functional.cross_entropy) to train this network.

We can write a function that iterates through the training set (via `train_loader`) and train for 1 epoch.

The next exercise is to fill in the code below. You can use the following pytorch functions:

* put data on GPU: [to](https://pytorch.org/docs/stable/generated/torch.Tensor.to.html?highlight=#torch.to)
* clear gradient: [zero_grad](https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.zero_grad.html?highlight=zero_grad)
* backward pass: [backward](https://pytorch.org/docs/stable/generated/torch.autograd.backward.html?highlight=backward#torch.autograd.backward)
* update parameters with a gradient step: [step](https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.step.html?highlight=step#torch.optim.Optimizer.step)





In [16]:
########################
#                      #
#       Exercise       #
#                      #
########################

# Fix the places with a `FIXME!!!`

def train(net, optim):
    net.train()
    for image, label in train_loader:
      # put data onto GPU
      img = image.cuda()
      lab = label.cuda()

      # clear gradient
      net.zero_grad()

      # forward through the network
      output = net.forward(img)

      # compute loss and gradient
      loss = F.cross_entropy(output, lab)
      loss.backward()

      # update parameters
      optim.step()

Let's also write a function that evaluates our network on the validation set.

In [17]:
########################
#                      #
#       Exercise       #
#                      #
########################

# Fix the places with a `FIXME!!!`


def evaluate(net):
    total = 0
    correct = 0
    
    net.eval()  # puts the network in eval mode. this is important when the 
                # network has layers that behaves differently in training and 
                # evaluation time, e.g., dropout and batch norm.
    for image, label in val_loader:
        # put data onto GPU
        img = image.cuda()
        label = label.cuda()
        
        with torch.no_grad():  # gradients are not tracked in this context manager
                               # since we are evaluating, gradients are not needed 
                               # and we can save some time and GPU memory.
              
            # forward through the network, and get the predicted class
            # FIXME!!!  (HINT: use .argmax(dim=-1))
            #   `prediction` should be an integer vector of size equal to the batch size.
            #   Remember that the network outputs logits of the prediction probabilities, 
            #   and that the higher the logits, the higher the probability.
            prediction = net.forward(img).argmax(dim=-1)
            
            total += image.size(0)  # batch size
            correct += (prediction == label).sum().item()  # `.item()` retreives a python number from a 1-element tensor
            
    return correct / total

In [18]:
# Without any training, the network accuracy matches that of random guessing: ~10%.

print('At initialization, the network has accuracy {:.4f}%'.format(evaluate(net) * 100))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


At initialization, the network has accuracy 10.7200%


### Putting Everything Together

In [20]:
########################
#                      #
#       Exercise       #
#                      #
########################


# Fix the places with a `FIXME!!!
from torch.optim import Adam


num_epochs = 10
lr = 0.01

optim = Adam(net.parameters(), lr=0.001)  # FIXME!!! Construct an optimizer (e.g., SGD) to optimize the network parameters with the above learning rate

for epoch in range(num_epochs):
    print('Epoch: {}\tValidation Accuracy: {:.4f}%'.format(epoch, evaluate(net) * 100))
    train(net, optim)

print('Done! \tValidation Accuracy: {:.4f}%'.format(evaluate(net) * 100))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 0	Validation Accuracy: 98.8500%
Epoch: 1	Validation Accuracy: 99.1900%
Epoch: 2	Validation Accuracy: 99.2100%
Epoch: 3	Validation Accuracy: 99.2100%
Epoch: 4	Validation Accuracy: 99.2700%
Epoch: 5	Validation Accuracy: 99.2600%
Epoch: 6	Validation Accuracy: 99.2400%
Epoch: 7	Validation Accuracy: 99.2500%
Epoch: 8	Validation Accuracy: 99.2600%
Epoch: 9	Validation Accuracy: 99.3100%
Done! 	Validation Accuracy: 99.2200%


## Open-ended Exercises (not required)

1. What could you do to make the accuracy higher and/or the training faster?  
2. Adapt the code for another dataset (e.g., CIFAR-10 from [`torchvision.datasets`](https://pytorch.org/docs/stable/torchvision/datasets.html)).
3. How dose your MNIST perform on [the USPS dataset](https://gist.github.com/SsnL/300865932b090d3f0798c2b961d371f9) (another black-white digits dataset)? Why?
4. How many samples are actually needed to train a good MNIST classifier? Try subsample the training set (sampled a fraction of images per class) and plot validation accuracy vs. training set size.
5. Use the [fast-gradient sign method](https://arxiv.org/abs/1412.6572) to attack your MNIST classifier! Why does your super great classifier now fail at a seemingly normal digit image?

In [62]:
# Fast-Gradient Sign Attack example

import PIL

# Utility function
def tensor2image(tensor):
    return PIL.Image.fromarray((tensor.detach() * 127.5 + 128).clamp_(0, 255).to('cpu', torch.uint8).numpy()[0])


In [63]:
# Get a data sample we want to attack

data, label = mnist_val[245]
data = data.to(cuda0)

In [64]:
# Look at the image

tensor2image(data)

In [65]:
# Label is...

print('label is', label)

label is 3


In [66]:
# Let's see what our trained network says about this image.


# turn the network into eval mode
net.eval()
output = net(data.unsqueeze(0))  # .unsqueeze(0) insert a batch dimension so it looks like batched data
prediction = output.argmax(-1)
print('predicted class is', prediction.item())
assert prediction.item() == label

predicted class is 3


In [67]:
# For fast-gradient sign attack to work, we need to get the gradient w.r.t. the input image. 


# Let's make it accept gradient first.
data = data.requires_grad_()

In [68]:
# Compute the loss as usual
loss = F.cross_entropy(net(data.unsqueeze(0)), torch.tensor([label], device=cuda0))

# backprop, now to `data` as well
loss.backward()

In [69]:
eps = 0.15

with torch.no_grad():
    # Fast-gradient sign attack (aka gradient **ascent** for loss maximization)
    new_data = (data + data.grad.sign() * eps).clamp_(-1, 1)  
    new_prediction = net(new_data.unsqueeze(0)).argmax(-1)
    print('The network predicts the attacked image as', new_prediction.item())

The network predicts the attacked image as 2


In [70]:
# visualize the attacked image

tensor2image(new_data)

In [71]:
# It still looks like a 3! But our network is now fooled...

Questions: 
+ Why does this happen? 
+ This method attacks the network so it predicts wrongly for this 3 image. Can you attack the network so it outputs a particular target class?